In [5]:
import talib
# import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5

In [12]:
class BackTest:
    
    def __init__(self, instrument):
        
#         self.df = pd.DataFrame(yf.download(tickers=instrument, start = '2022-10-01', end = '2023-06-01', interval='1h'))
        if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
        login = 114781990
        password = 'Mypassword$1234'
        server = 'Exness-MT5Trial6'

        mt5.login(login, password, server)
        print('Working on: ', instrument[0])
        rate = mt5.copy_rates_from(instrument[0], mt5.TIMEFRAME_M15, datetime.datetime.now(), instrument[2])
        self.nf = pd.DataFrame(rate)
        
         
        self.nf['date']=pd.to_datetime(self.nf['time'], unit='s')

        '''Renaming columns and deleting unnecessary columns'''

        self.nf.set_index(np.arange(len(self.nf)), inplace = True)
        self.nf = self.nf.drop(['spread', 'real_volume', 'tick_volume'], axis='columns')
        
        
#         for trend
        self.position_index = 0
        
        # Calculate the number of rows in each DataFrame
#         n = len(self.nf)
#         n_half = n // 2  # Calculate the half point

        # Split the DataFrame into two equal parts
        self.df = self.nf.iloc[:500]  # First half
        self.df2 = self.nf.iloc[500:]  # Second half
        
#         self.df.to_excel('check1.xlsx')
#         self.df2.to_excel('check2.xlsx')

        
    def display(self, name):
        
        self.df.to_excel(f'{name}.xlsx')
        
        
    def define_trend(self, current_high, current_low, prev_high, prev_low, days, i):
        #       Uptrend
            if current_high > prev_high and current_low > prev_low:
    #           Find point at which current sample
    #           crossed prev high

                stream = self.df.loc[self.df.index[i : i+days], 'high']
                up_candle_index = stream.loc[stream > prev_high].index[0]
                
    #             if df.loc[i+15, 'high'] > prev_high:
                    
                self.df.loc[up_candle_index : i+days, 'trend'] = 'up'

            
                if self.df.iloc[i-1]['trend'] == 'up':
                    self.df.loc[i: up_candle_index, 'trend'] = 'up'

                else:
                    self.df.loc[i: up_candle_index, 'trend'] = 'no trend'


    #         Downtrend
            elif (current_high < prev_high) and (current_low < prev_low):
    #           Find point at which current sample 
    #           crossed prev low

                stream = self.df.loc[self.df.index[i : i+days], 'low']
                down_candle_index = stream.loc[stream < prev_low,].index[0]
               
#                 print('DownCandleIndex: ', down_candle_index)
#                 print('Days Callee: ', days)
                
                self.df.loc[down_candle_index : i+days, 'trend'] = 'down'

                if self.df.iloc[i-1]['trend'] == 'down':
                    self.df.loc[i: down_candle_index, 'trend'] = 'down'

                else:
                    self.df.loc[i: down_candle_index, 'trend'] = 'no trend'


                    
            else:
                self.df.loc[i : i+days, 'trend'] = 'no trend'

    
    def get_trend(self, i=0):
        
        length = len(self.df)

        # print('Length: ', length)
        
        days = 21

        prev_high = self.df.loc[self.df.index[i : i+days], 'high'].max()
        
        prev_low = self.df.loc[self.df.index[i : i+days], 'low'].min()

        while(i + days  < length):

            # print('In Trend !!')
        
            i += days
            
            current_high = self.df.loc[self.df.index[i : i+days], 'high'].max()
            current_low = self.df.loc[self.df.index[i : i+days], 'low'].min()
            
            self.define_trend(current_high, current_low, prev_high, prev_low, days, i)
            
            prev_high = current_high
            prev_low = current_low

            
                    
                
        if i < length-1:
            
            prev_high = self.df.loc[self.df.index[i-days:i], 'high'].max()
            prev_low = self.df.loc[self.df.index[i-days:i], 'low'].min()
            
            current_high = self.df.loc[self.df.index[i : length], 'high'].max()
            current_low = self.df.loc[self.df.index[i : length], 'low'].min()
            
            days = length-i
            
            self.define_trend(current_high, current_low, prev_high, prev_low, days, i)
            
#       Marking position of last candle with 21 group
        self.position_index = i
#         print('Past Position: ',self.position_index)
        self.prev_high = current_high
        self.prev_low = current_low
        
        
    # KELTNER CHANNEL CALCULATION

    def get_kc(self, high, low, close, kc_lookback, multiplier, atr_lookback):
        tr1 = pd.DataFrame(high - low)
        tr2 = pd.DataFrame(abs(high - close.shift()))
        tr3 = pd.DataFrame(abs(low - close.shift()))
        
        # Calculate True Range as the element-wise maximum of the three components
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.ewm(alpha=1/atr_lookback).mean()

        kc_middle = close.ewm(span=kc_lookback).mean()
        kc_upper = kc_middle + multiplier * atr
        kc_lower = kc_middle - multiplier * atr

        return kc_middle, kc_upper, kc_lower
    
    
    def in_squeeze(self, df):
        return df['lower_band'] > df['lower_keltner'] and df['upper_band'] < df['upper_keltner']
    
    def indicators(self):
        '''Adding EMAs, BB, Keltner Channel and RSI'''
        
        self.df['EMA_8'] = talib.EMA(self.df['close'], timeperiod=8)


        self.df['EMA_20'] = talib.EMA(self.df['close'], timeperiod=20)

        self.df['EMA_200'] = talib.EMA(self.df['close'], timeperiod=200)


        self.df['RSI'] = talib.RSI(self.df['close'], timeperiod=21)
        self.df = self.df.dropna()

        self.df.set_index(np.arange(len(self.df)), inplace = True)
        
        # Identify trend on 21 days highs & lows
        self.get_trend()


        '''Bollinger Band'''
        self.df['20sma'] = self.df['close'].rolling(window=20).mean()
        self.df['stddev'] = self.df['close'].rolling(window=20).std()
        self.df['lower_band'] = self.df['20sma'] - (2 * self.df['stddev'])
        self.df['upper_band'] = self.df['20sma'] + (2 * self.df['stddev'])

        '''Keltners Channel'''
        self.df['TR'] = abs(self.df['high'] - self.df['low'])
        self.df['ATR'] = self.df['TR'].rolling(window=20).mean()

        self.df['lower_keltner'] = self.df['20sma'] - (self.df['ATR'] * 1.2)
        self.df['upper_keltner'] = self.df['20sma'] + (self.df['ATR'] * 1.2)
        
        
        self.df['squeeze_on'] = self.df.apply(self.in_squeeze, axis=1)

        self.df = self.df.drop(['stddev', 'lower_band', 'upper_band', '20sma', 'TR', 'ATR', 'lower_keltner', 'upper_keltner'],axis=1)

        '''Get Keltner for trading'''
        
        self.df['EMA_20'],self.df['upper_keltner'],self.df['lower_keltner'] = self.get_kc(self.df['high'], self.df['low'], self.df['close'], kc_lookback = 20, multiplier = 2.25, atr_lookback = 10)
        
        self.df = self.df.dropna()

        self.df.set_index(np.arange(len(self.df)), inplace = True)

        self.df.index.name='sn'
        
        
    
#    Calculating the current candle trend
    def test_current_trend(self, high, low, l):
        
        if high > self.prev_high and low > self.prev_low:

            self.df.loc[l, 'trend'] = 'up'
            
            self.prev_low = self.df.loc[self.df.index[self.position_index : l+1], 'low'].min()

        elif high < self.prev_high and low < self.prev_low:

            self.df.loc[l, 'trend'] = 'down'
            
            self.prev_low = self.df.loc[self.df.index[self.position_index : l+1], 'high'].max()

        else:
            self.df.loc[l, 'trend'] = self.df.loc[l-1,'trend']
        
    
#   Calculating the current candle trend
    def current_trend(self, days=21):
        l = len(self.df) - 1
        
#       check for trend in current candle
#       when its smaller than next 21 days window
        if self.position_index + days > l:
            
            self.test_current_trend( self.df.iloc[l]['high'], self.df.iloc[l]['low'], l)
           
                
#         elif self.position_index + days < l:
            
#             self.prev_high = self.df.loc[self.df.index[self.position_index : l+1], 'high'].max()
#             self.prev_low = self.df.loc[self.df.index[self.position_index : l+1], 'low'].min()
            
#             self.test_current_trend( self.df.iloc[l]['high'], self.df.iloc[l]['low'], l)
            
            
        #         Move position_index to 21 days window
#         For No-trend compare this high and low to prev.
        elif self.position_index + days == l:
            
            self.current_high = self.df.loc[self.df.index[self.position_index : l+1], 'high'].max()
            self.current_low = self.df.loc[self.df.index[self.position_index : l+1], 'low'].min()
            
            if self.current_high > self.prev_high and self.current_low > self.prev_low:

                self.df.loc[l, 'trend'] = 'up'

            elif self.current_high < self.prev_high and self.current_low < self.prev_low:

                self.df.loc[l, 'trend'] = 'down'

            else:
                self.df.loc[l, 'trend'] ='no trend'

            self.position_index = l
            self.prev_high = self.current_high
            self.prev_low = self.current_low
            
            
            
    def current_indicators(self):
        '''Adding EMAs, BB, Keltner Channel and RSI'''
        
        length = len(self.df) - 1

        num_data_points = 200

        # Slice the last 'num_data_points' rows of the DataFrame
        recent_data = self.df.iloc[length - num_data_points + 1 : length + 1]

        # Calculate indicators for the sliced data
        recent_data['EMA_8'] = talib.EMA(recent_data['close'], timeperiod=8)
        recent_data['EMA_20'] = talib.EMA(recent_data['close'], timeperiod=20)
        recent_data['EMA_200'] = talib.EMA(recent_data['close'], timeperiod=200)
        recent_data['RSI'] = talib.RSI(recent_data['close'], timeperiod=21)

        # Identify trend on 21 days highs & lows
        self.current_trend()


        '''Bollinger Band'''
        self.df.loc[length,'20sma'] = self.df['close'][-20:].rolling(window=20).mean()
        self.df.loc[length,'stddev'] = self.df['close'][-20:].rolling(window=20).std()
        self.df.loc[length,'lower_band'] = self.df['20sma'] - (2 * self.df['stddev'])
        self.df.loc[length,'upper_band'] = self.df['20sma'] + (2 * self.df['stddev'])

        '''Keltners Channel'''
        self.df.loc[length,'TR'] = abs(self.df['high'] - self.df['low'])
        self.df.loc[length,'ATR'] = self.df['TR'][-20:].rolling(window=20).mean()

        self.df.loc[length,'lower_keltner'] = self.df['20sma'] - (self.df['ATR'] * 1.2)
        self.df.loc[length,'upper_keltner'] = self.df['20sma'] + (self.df['ATR'] * 1.2)
        
        
        self.df['squeeze_on'] = self.df.apply(self.in_squeeze, axis=1)

        self.df = self.df.drop(['stddev', 'lower_band', 'upper_band', '20sma', 'TR', 'ATR', 'lower_keltner', 'upper_keltner'],axis=1)

        '''Get Keltner for trading'''
        
        self.df['EMA_20'],self.df['upper_keltner'],self.df['lower_keltner'] = self.get_kc(self.df['high'][-20:], self.df['low'][-20:], self.df['close'][-20:], kc_lookback = 20, multiplier = 2.25, atr_lookback = 10)
        
        self.df = self.df.dropna()

        self.df.set_index(np.arange(len(self.df)), inplace = True)

        self.df.index.name='sn'
            
            
    def get_current_data(self):
        
        for index, row in self.df2.iterrows():
            self.df = self.df.append(row, ignore_index=True)

            # Call the custom function after each append
            self.current_indicators()
            

In [13]:
b = BackTest(['EURUSDm',10, 24000])

b.indicators()
# b.display('past')
b.current_indicators()
b.display('current')

Working on:  EURUSDm


C:\Users\hp\AppData\Local\Temp\ipykernel_10088\624063329.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['EMA_8'] = talib.EMA(self.df['close'], timeperiod=8)
C:\Users\hp\AppData\Local\Temp\ipykernel_10088\624063329.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['EMA_20'] = talib.EMA(self.df['close'], timeperiod=20)
C:\Users\hp\AppData\Local\Temp\ipykernel_10088\624063329.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

ValueError: Incompatible indexer with Series

In [ ]:
# calculating only 200 candles and merging them

def current_indicators(self):
    '''Adding EMAs, BB, Keltner Channel, and RSI'''

    length = len(self.df) - 1

    num_data_points = 200

    # Calculate indicators for the recent data (last 'num_data_points' rows)
    recent_data = self.df.iloc[length - num_data_points + 1 : length + 1].copy()  # Create a copy to avoid modifying the original DataFrame

    recent_data['EMA_8'] = talib.EMA(recent_data['close'], timeperiod=8)
    recent_data['EMA_20'] = talib.EMA(recent_data['close'], timeperiod=20)
    recent_data['EMA_200'] = talib.EMA(recent_data['close'], timeperiod=200)
    recent_data['RSI'] = talib.RSI(recent_data['close'], timeperiod=21)

    # Identify trend on 21 days highs & lows
    self.current_trend()

    '''Bollinger Band'''
    recent_data['20sma'] = recent_data['close'].rolling(window=20).mean()
    recent_data['stddev'] = recent_data['close'].rolling(window=20).std()
    recent_data['lower_band'] = recent_data['20sma'] - (2 * recent_data['stddev'])
    recent_data['upper_band'] = recent_data['20sma'] + (2 * recent_data['stddev'])

    '''Keltners Channel'''
    recent_data['TR'] = abs(recent_data['high'] - recent_data['low'])
    recent_data['ATR'] = recent_data['TR'].rolling(window=20).mean()

    recent_data['lower_keltner'] = recent_data['20sma'] - (recent_data['ATR'] * 1.2)
    recent_data['upper_keltner'] = recent_data['20sma'] + (recent_data['ATR'] * 1.2)

    recent_data['squeeze_on'] = recent_data.apply(self.in_squeeze, axis=1)

    # Drop unnecessary columns from recent_data
    recent_data = recent_data.drop(['stddev', 'lower_band', 'upper_band', '20sma', 'TR', 'ATR', 'lower_keltner', 'upper_keltner'], axis=1)

    '''Get Keltner for trading'''
    recent_data['EMA_20'], recent_data['upper_keltner'], recent_data['lower_keltner'] = self.get_kc(
        recent_data['high'], recent_data['low'], recent_data['close'], kc_lookback=20, multiplier=2.25, atr_lookback=10
    )

    # Concatenate recent_data with the original DataFrame
    self.df = pd.concat([self.df, recent_data], axis=0, ignore_index=True)

    self.df.dropna(inplace=True)  # Drop any NaN rows

    self.df.set_index(np.arange(len(self.df)), inplace=True)
    self.df.index.name = 'sn'


In [ ]:
print('In Sell')

length = len(self.df) - 1


current_candle = self.df.loc[length]['candle']
                
if current_candle == 'R':
    # TRG  and Trail SL condition

    if self.df.loc[length]['low'] <= self.target:
        # TRG

        self.result.loc[self.result_len, 'EXIT'] = self.target
        self.result.loc[self.result_len ,'EXIT DATE'] = self.df.loc[length]['date']
        self.result.loc[self.result_len, 'REASON'] = 'TRG'
        self.sell_on = False
        self.trail_sl = False

 

    # For Breakeven
    elif self.df.loc[length]['low'] <= self.breakeven:
        self.sl = min(self.df.loc[self.base_candle_index]['close'], self.df.loc[length]['EMA_8'])
        self.trail_sl = True


#               For Gap up exit
    elif self.sl <= self.df.loc[length]['open']:

        self.result.loc[self.result_len, 'EXIT'] =  self.df.loc[length]['open']
        self.result.loc[self.result_len, 'EXIT DATE'] = self.df.loc[length]['date']
        
        if self.trail_sl:
            self.result.loc[self.result.loc[self.result_len, 'REASON'] = 'TRAIL_SL'
            self.trail_sl = False
            
        else:
            self.result.loc[self.result.loc[self.result_len, 'REASON'] = 'SL'
                            
        self.sell_on = False
        

    # Red candle High hits SL 
    elif self.sl <= self.df.loc[length]['high']:

        self.result.loc[self.result_len, 'EXIT'] =  self.sl
        self.result.loc[self.result_len, 'EXIT DATE'] = self.df.loc[length]['date']
        
        if self.trail_sl:
            self.result.loc[self.result.loc[self.result_len, 'REASON'] = 'TRAIL_SL'
            self.trail_sl = False
            
        else:
            self.result.loc[self.result_len, 'REASON'] = 'SL'
    
        self.sell_on = False
        

elif current_candle == 'G':
# SL condition

#                 For Gap UP
    if self.sl <= self.df.loc[length]['open']:

        self.result.loc[self.result_len, 'EXIT'] =  self.df.loc[length]['open']
        self.result.loc[self.result_len, 'EXIT DATE'] = self.df.loc[length]['date']
        
        if self.trail_sl:
            self.result.loc[self.result_len, 'REASON'] = 'TRAIL_SL'
            self.trail_sl = False
        
        else:
            self.result.loc[self.result_len, 'REASON'] = 'SL'

        self.sell_on = False

#                 For SL
    elif self.sl <= self.df.loc[length]['high']:

        self.result.loc[self.result_len, 'EXIT'] = self.sl
        self.result.loc[self.result_len, 'EXIT DATE'] = self.df.loc[length]['date']
        if self.trail_sl:
            self.result.loc[self.result_len, 'REASON'] = 'TRAIL_SL'
            self.trail_sl = False
        
        else:
            self.result.loc[self.result_len, 'REASON'] = 'SL'

        self.sell_on = False


#  Max wait period 11 Days till breakeven 
# or current candle closed above 1% to last
# else exit the trade
prev_close_one_percent = (self.df.loc[length - 1]['close'])/100

if ((length > (self.base_candle_index + 10)) and (self.df.loc[length]['close'] > self.breakeven) and
    (self.df.loc[length]['close'] >  (self.df.loc[length - 1]['close']) - prev_close_one_percent)):

    self.result.loc[self.result_len, 'EXIT'] = self.df.loc[length]['close']
    self.result.loc[self.result_len ,'EXIT DATE'] = self.df.loc[length]['date']
    self.result.loc[self.result_len, 'REASON'] = 'NO MOVE'
    self.sell_on = False
    self.trail_sl = False
